In [32]:
import pandas as pd
import numpy as np

# ============================================================================
# CARGA DE DATOS
# ============================================================================

print("📊 Cargando archivos de datos...")

# Cargar TopRepositorios
df_top_repos = pd.read_csv('../Datos/TopRepositorios.csv')
print(f"✓ TopRepositorios cargado: {len(df_top_repos)} registros")

# Cargar TopReposXLenguajes.txt
df_repos_lang = pd.read_csv('../Datos/TopReposXLenguajes.csv')
print(f"✓ TopReposXLenguajes cargado: {len(df_repos_lang)} registros")

📊 Cargando archivos de datos...
✓ TopRepositorios cargado: 13 registros
✓ TopReposXLenguajes cargado: 188 registros


In [33]:
# ============================================================================
# LIMPIEZA DE DATOS
# ============================================================================

def limpiar_numeros(df, columnas):
    """
    Convierte columnas con números formateados (e.g., '6,702') a enteros
    """
    for col in columnas:
        df[col] = df[col].str.replace(',', '').str.replace('"', '').astype(int)
    return df

print("\n🧹 Limpiando datos...")

# Limpiar números en TopRepositorios
df_top_repos = limpiar_numeros(df_top_repos, ['NumberOfStar', 'NumberOfFork'])

# Limpiar números en TopReposXLenguajes
df_repos_lang = limpiar_numeros(df_repos_lang, ['NumberOfStar', 'NumberOfFork'])

print("✓ Números convertidos a formato numérico")


🧹 Limpiando datos...
✓ Números convertidos a formato numérico


In [34]:
# ============================================================================
# ESTADÍSTICAS GENERALES
# ============================================================================

print("\n📈 Estadísticas Generales:")
print(f"  - Total de lenguajes únicos: {df_repos_lang['Language'].nunique()}")
print(f"  - Repositorio más popular: {df_top_repos.loc[df_top_repos['NumberOfStar'].idxmax(), 'Repository']}")
print(f"  - Total de estrellas (top repos): {df_top_repos['NumberOfStar'].sum():,}")
print(f"  - Total de forks (top repos): {df_top_repos['NumberOfFork'].sum():,}")


📈 Estadísticas Generales:
  - Total de lenguajes únicos: 10
  - Repositorio más popular: opencode
  - Total de estrellas (top repos): 323,404
  - Total de forks (top repos): 37,031


In [35]:
# ============================================================================
# ANÁLISIS POR LENGUAJE
# ============================================================================

print("\n🔍 Análisis por Lenguaje:")

# Estadísticas agregadas por lenguaje
stats_por_lenguaje = df_repos_lang.groupby('Language').agg({
    'NumberOfStar': ['sum', 'mean', 'count'],
    'NumberOfFork': ['sum', 'mean']
}).round(0)

stats_por_lenguaje.columns = ['Total_Stars', 'Promedio_Stars', 'Num_Repos', 'Total_Forks', 'Promedio_Forks']
stats_por_lenguaje = stats_por_lenguaje.reset_index()
stats_por_lenguaje = stats_por_lenguaje.sort_values('Total_Stars', ascending=False)

print("\nTop 10 Lenguajes por Total de Estrellas:")
print(stats_por_lenguaje.head(10))


🔍 Análisis por Lenguaje:

Top 10 Lenguajes por Total de Estrellas:
     Language  Total_Stars  Promedio_Stars  Num_Repos  Total_Forks  \
7        Rust       771430         33540.0         23        69092   
1         C++       521261         26063.0         20        92765   
2          Go       450283         23699.0         19        62050   
9  TypeScript       437436         33649.0         13        55393   
4         PHP       396125         16505.0         24        90444   
3  JavaScript       352912         25208.0         14        84735   
6        Ruby       303259         13784.0         22        93900   
0           C       270122         14217.0         19        41762   
8       Swift       208181          9463.0         22        26583   
5      Python       206752         17229.0         12        24245   

   Promedio_Forks  
7          3004.0  
1          4638.0  
2          3266.0  
9          4261.0  
4          3768.0  
3          6052.0  
6          4268.0  
0

In [36]:
# ============================================================================
# PREPARAR DATOS PARA VISUALIZACIÓN
# ============================================================================

print("\n💾 Preparando datos para visualización...")

# 1. Promedio de estrellas por lenguaje (para gráfico de barras)
promedio_estrellas = stats_por_lenguaje[['Language', 'Promedio_Stars', 'Num_Repos']].copy()
promedio_estrellas = promedio_estrellas.sort_values('Promedio_Stars', ascending=False).head(10)

# 2. Top repositorios por lenguaje específico (para análisis detallado)
top_repos_por_lang = {}
for lang in df_repos_lang['Language'].unique():
    repos = df_repos_lang[df_repos_lang['Language'] == lang].nlargest(5, 'NumberOfStar')
    top_repos_por_lang[lang] = repos

# 3. Distribución de repos por lenguaje
distribucion_lenguajes = df_repos_lang['Language'].value_counts().reset_index()
distribucion_lenguajes.columns = ['Language', 'Count']


💾 Preparando datos para visualización...


In [37]:
# ============================================================================
# EXPORTAR DATOS PROCESADOS
# ============================================================================

print("\n📁 Guardando datos procesados...")

# Crear directorio si no existe
import os
os.makedirs('datos_procesados', exist_ok=True)

# Guardar archivos procesados
df_top_repos.to_csv('../Datos_procesados/Top_repos_clean.csv', index=False)
df_repos_lang.to_csv('../Datos_procesados/Repos_por_lenguaje_clean.csv', index=False)
stats_por_lenguaje.to_csv('../Datos_procesados/Estadisticas_lenguajes.csv', index=False)
promedio_estrellas.to_csv('../Datos_procesados/Promedio_estrellas_top10.csv', index=False)
distribucion_lenguajes.to_csv('../Datos_procesados/Distribucion_lenguajes.csv', index=False)

print("✓ Archivos guardados en carpeta 'datos_procesados/'")
print("\n✅ Procesamiento completado exitosamente!")


📁 Guardando datos procesados...
✓ Archivos guardados en carpeta 'datos_procesados/'

✅ Procesamiento completado exitosamente!


In [ ]:
# ============================================================================
# VISTA PREVIA DE DATOS
# ============================================================================

print("\n👀 Vista previa de datos procesados:")
print("\n--- TopRepositorios (primeras 5 filas) ---")
print(df_top_repos.head())

print("\n--- Estadísticas por Lenguaje (Top 5) ---")
print(stats_por_lenguaje.head())

print("\n--- Distribución de Lenguajes ---")
print(distribucion_lenguajes.head(10))